### importing libaries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from IPython.display import display

### reading data set

In [2]:
data = pd.read_csv("covtype.data",header=None)
data.columns = ["Elevation","Aspect","Slope","HDH","VDH","HDR","Hillshade_9am","Hillshade_Noon","Hillshade_3pm","HDFP","Wilderness_Area Rawah","Wilderness_Area Neota","Wilderness_Area Comanche_Peak","Wilderness_Area Cache_la_Poudre","Soil_Type USFS2702","Soil_Type USFS2703","Soil_Type USFS2704","Soil_Type USFS2705","Soil_Type USFS2706","Soil_Type USFS2717","Soil_Type USFS3501","Soil_Type USFS3502","Soil_Type USFS4201","Soil_Type USFS4703","Soil_Type USFS4704","Soil_Type USFS4744","Soil_Type USFS4758","Soil_Type USFS5101","Soil_Type USFS5151","Soil_Type USFS6101","Soil_Type USFS6102","Soil_Type USFS6731","Soil_Type USFS7101","Soil_Type USFS7102","Soil_Type USFS7103","Soil_Type USFS7201","Soil_Type USFS7202","Soil_Type USFS7700","Soil_Type USFS7701","Soil_Type USFS7702","Soil_Type USFS7709","Soil_Type USFS7710","Soil_Type USFS7745","Soil_Type USFS7746","Soil_Type USFS7755","Soil_Type USFS7756","Soil_Type USFS7757","Soil_Type USFS7790","Soil_Type USFS8703","Soil_Type USFS8707","Soil_Type USFS8708","Soil_Type USFS8771","Soil_Type USFS8772","Soil_Type USFS8776","Cover_Type"]
print(data.shape)
display(data)
# display(data.head(10).T)

(581012, 55)


,Elevation,Aspect,Slope,HDH,VDH,HDR,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,HDFP,...,Soil_Type USFS7756,Soil_Type USFS7757,Soil_Type USFS7790,Soil_Type USFS8703,Soil_Type USFS8707,Soil_Type USFS8708,Soil_Type USFS8771,Soil_Type USFS8772,Soil_Type USFS8776,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,0,0,3
581008,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,0,0,3
581009,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,0,0,3
581010,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,0,0,3


#### this dataset already make 2 col one-hotted so for not repeating a single operation 44 times I did revert it

##### I copied this piece of code from Stack overflow

In [3]:
def undummify(df : pd.DataFrame, prefix_sep=" ") -> pd.DataFrame:
    cols2collapse = {
        item.split(prefix_sep)[0]: (prefix_sep in item) for item in df.columns
    }
    series_list = []
    for col, needs_to_collapse in cols2collapse.items():
        if needs_to_collapse:
            undummified = (
                df.filter(like=col)
                .idxmax(axis=1)
                .apply(lambda x: x.split(prefix_sep, maxsplit=1)[1])
                .rename(col)
            )
            series_list.append(undummified)
        else:
            series_list.append(df[col])
    undummified_df = pd.concat(series_list, axis=1)
    return undummified_df


In [4]:
display(undummify(data).sample(15,random_state=10).T)
data = undummify(data)

,464108,471404,545113,128748,373677,436787,398464,181326,195134,73676,150530,474652,369237,386489,140891
Elevation,3381,3039,3318,2823,3211,2797,3385,3065,3209,3004,2898,2925,3153,3264,2985
Aspect,0,291,353,67,16,129,282,0,157,151,124,5,30,113,320
Slope,18,12,21,13,19,9,8,4,7,18,5,28,12,7,16
HDH,497,361,309,170,350,0,899,242,309,90,335,150,216,210,120
VDH,187,87,84,54,36,0,179,10,46,3,65,58,9,31,4
HDR,4201,618,4548,2514,3254,2400,1092,3298,5057,3458,2989,272,3072,1950,3837
Hillshade_9am,190,187,179,233,200,235,198,214,228,240,229,172,216,233,177
Hillshade_Noon,203,238,198,213,196,235,241,232,241,237,236,173,214,233,223
Hillshade_3pm,147,191,153,110,128,130,182,156,146,120,140,127,132,132,185
HDFP,3736,742,1324,2879,3129,1611,2130,485,1575,5949,2230,162,3150,1399,1521


In [5]:
df = data.sample(frac=0.2, random_state=1337)
train_df = df.sample(frac=0.7, random_state=1337)
val_df = df.drop(train_df.index)
test_df = data.drop(df.index)
print(df.shape, train_df.shape ,val_df.shape ,test_df.shape)

(116202, 13) (81341, 13) (34861, 13) (464810, 13)


#### turning Pandas_DataFrame to TF_DataSet

In [6]:
def dataframe_to_dataset(dataframe:pd.DataFrame) -> tf.data.Dataset:
    dataframe = dataframe.copy()
    labels = dataframe.pop("Cover_Type")
    labels = keras.utils.to_categorical(labels)
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))

    return ds

In [7]:
train_ds = dataframe_to_dataset(train_df)
val_ds = dataframe_to_dataset(val_df)
test_ds = dataframe_to_dataset(test_df)

In [8]:
for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)

Input: {'Elevation': <tf.Tensor: shape=(), dtype=int64, numpy=3002>, 'Aspect': <tf.Tensor: shape=(), dtype=int64, numpy=356>, 'Slope': <tf.Tensor: shape=(), dtype=int64, numpy=19>, 'HDH': <tf.Tensor: shape=(), dtype=int64, numpy=127>, 'VDH': <tf.Tensor: shape=(), dtype=int64, numpy=15>, 'HDR': <tf.Tensor: shape=(), dtype=int64, numpy=430>, 'Hillshade_9am': <tf.Tensor: shape=(), dtype=int64, numpy=186>, 'Hillshade_Noon': <tf.Tensor: shape=(), dtype=int64, numpy=203>, 'Hillshade_3pm': <tf.Tensor: shape=(), dtype=int64, numpy=152>, 'HDFP': <tf.Tensor: shape=(), dtype=int64, numpy=1222>, 'Wilderness_Area': <tf.Tensor: shape=(), dtype=string, numpy=b'Comanche_Peak'>, 'Soil_Type': <tf.Tensor: shape=(), dtype=string, numpy=b'USFS7756'>}
Target: tf.Tensor([0. 0. 1. 0. 0. 0. 0. 0.], shape=(8,), dtype=float32)


In [9]:
train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)
test_ds = val_ds.batch(32)

### Configuring a FeatureSpace

In [10]:
from keras.utils import FeatureSpace

feature_space = FeatureSpace(
    features={
        # Categorical features encoded as integers
        # "sex": FeatureSpace.integer_categorical(num_oov_indices=0),
        # Categorical feature encoded as string
        "Wilderness_Area": FeatureSpace.string_categorical(num_oov_indices=0),
        "Soil_Type": FeatureSpace.string_categorical(num_oov_indices=0),
        # Numerical features to normalize
        # "age": FeatureSpace.float_discretized(num_bins=30),
        # Numerical features to normalize
        "Elevation": FeatureSpace.float_normalized(),
        "Aspect": FeatureSpace.float_normalized(),
        "Slope": FeatureSpace.float_normalized(),
        "HDH": FeatureSpace.float_normalized(),
        "VDH": FeatureSpace.float_normalized(),
        "HDR": FeatureSpace.float_normalized(),
        "Hillshade_9am": FeatureSpace.float_normalized(),
        "Hillshade_Noon": FeatureSpace.float_normalized(),
        "Hillshade_3pm": FeatureSpace.float_normalized(),
        "HDFP": FeatureSpace.float_normalized(),

    },
    # Specify feature cross with a custom crossing dim.
    # crosses=[
    #     FeatureSpace.cross(feature_names=("Soil_Type", "Wilderness_Area"), crossing_dim=32),
    # ],
    output_mode="concat",
)

#### Adapt that Config to train_ds

In [11]:
train_ds_with_no_labels = train_ds.map(lambda x, _: x)
feature_space.adapt(train_ds_with_no_labels)

In [12]:
for x, _ in train_ds.take(1):
    preprocessed_x = feature_space(x)
    print("preprocessed_x.shape:", preprocessed_x.shape)
    print("preprocessed_x.dtype:", preprocessed_x.dtype)

preprocessed_x.shape: (32, 53)
preprocessed_x.dtype: <dtype: 'float32'>


#### Assigning Var to each transformed DSs

In [13]:
preprocessed_train_ds = train_ds.map(
    lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
preprocessed_train_ds = preprocessed_train_ds.prefetch(tf.data.AUTOTUNE)

preprocessed_val_ds = val_ds.map(
    lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
preprocessed_val_ds = preprocessed_val_ds.prefetch(tf.data.AUTOTUNE)

preprocessed_test_ds = test_ds.map(
    lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
preprocessed_test_ds = preprocessed_test_ds.prefetch(tf.data.AUTOTUNE)

### ANNs Modelling

##### My first and reference model which I made luckily and used model from Keras doc, and add 3 more layers to it which got 83% accuracy

In [14]:
dict_inputs = feature_space.get_inputs()
encoded_features = feature_space.get_encoded_features()

x = keras.layers.Dense(16, activation="relu")(encoded_features)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(32, activation="relu")(x)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(64, activation="relu")(x)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(32, activation="relu")(x)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(16, activation="relu")(x)
# x = keras.layers.Dropout(0.3)(x)
predictions = keras.layers.Dense(8, activation="softmax")(x)

training_model = keras.Model(inputs=encoded_features, outputs=predictions)
training_model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

inference_model = keras.Model(inputs=dict_inputs, outputs=predictions)



training_model.fit(
    preprocessed_train_ds, epochs=25, validation_data=preprocessed_val_ds, verbose=1
)

Epoch 1/25
2542/2542 [==============================] - 4s 1ms/step - loss: 0.7023 - accuracy: 0.7053 - val_loss: 0.6277 - val_accuracy: 0.7236
Epoch 2/25
2542/2542 [==============================] - 3s 1ms/step - loss: 0.5989 - accuracy: 0.7445 - val_loss: 0.5788 - val_accuracy: 0.7509
Epoch 3/25
2542/2542 [==============================] - 3s 1ms/step - loss: 0.5648 - accuracy: 0.7581 - val_loss: 0.5848 - val_accuracy: 0.7467
Epoch 4/25
2542/2542 [==============================] - 3s 1ms/step - loss: 0.5426 - accuracy: 0.7653 - val_loss: 0.5381 - val_accuracy: 0.7700
Epoch 5/25
2542/2542 [==============================] - 3s 1ms/step - loss: 0.5238 - accuracy: 0.7756 - val_loss: 0.5385 - val_accuracy: 0.7683
Epoch 6/25
2542/2542 [==============================] - 3s 1ms/step - loss: 0.5086 - accuracy: 0.7820 - val_loss: 0.5138 - val_accuracy: 0.7801
Epoch 7/25
2542/2542 [==============================] - 3s 1ms/step - loss: 0.4948 - accuracy: 0.7894 - val_loss: 0.5149 - val_accuracy:

##### In second one, I used dropout and deleted 64-nodes layer to accel its computing without adding more complex layers but accuracy decreased too much

In [15]:
dict_inputs = feature_space.get_inputs()
encoded_features = feature_space.get_encoded_features()

x = keras.layers.Dense(16, activation="relu")(encoded_features)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(32, activation="relu")(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(32, activation="relu")(x)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(16, activation="relu")(x)
# x = keras.layers.Dropout(0.3)(x)
predictions = keras.layers.Dense(8, activation="softmax")(x)

training_model = keras.Model(inputs=encoded_features, outputs=predictions)
training_model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

inference_model = keras.Model(inputs=dict_inputs, outputs=predictions)



training_model.fit(
    preprocessed_train_ds, epochs=20, validation_data=preprocessed_val_ds, verbose=1
)

Epoch 1/20
2542/2542 [==============================] - 4s 1ms/step - loss: 0.8148 - accuracy: 0.6693 - val_loss: 0.6581 - val_accuracy: 0.7204
Epoch 2/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.6709 - accuracy: 0.7228 - val_loss: 0.6833 - val_accuracy: 0.6899
Epoch 3/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.6452 - accuracy: 0.7326 - val_loss: 0.6514 - val_accuracy: 0.7041
Epoch 4/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.6271 - accuracy: 0.7407 - val_loss: 0.6308 - val_accuracy: 0.7287
Epoch 5/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.6176 - accuracy: 0.7432 - val_loss: 0.6513 - val_accuracy: 0.7187
Epoch 6/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.6086 - accuracy: 0.7468 - val_loss: 0.6656 - val_accuracy: 0.7124
Epoch 7/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.6020 - accuracy: 0.7515 - val_loss: 0.6342 - val_accuracy:

##### In 3rd one, I added Dropout layer but didn't delete any layer also got pretty good accuracy (82%)

In [16]:
dict_inputs = feature_space.get_inputs()
encoded_features = feature_space.get_encoded_features()

x = keras.layers.Dense(16, activation="relu")(encoded_features)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(32, activation="relu")(x)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(64, activation="relu")(x)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(32, activation="relu")(x)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(16, activation="relu")(x)
# x = keras.layers.Dropout(0.3)(x)
predictions = keras.layers.Dense(8, activation="softmax")(x)

training_model = keras.Model(inputs=encoded_features, outputs=predictions)
training_model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

inference_model = keras.Model(inputs=dict_inputs, outputs=predictions)



training_model.fit(
    preprocessed_train_ds, epochs=20, validation_data=preprocessed_val_ds, verbose=1
)

Epoch 1/20
2542/2542 [==============================] - 4s 1ms/step - loss: 0.7389 - accuracy: 0.6960 - val_loss: 0.6222 - val_accuracy: 0.7356
Epoch 2/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.6236 - accuracy: 0.7359 - val_loss: 0.5966 - val_accuracy: 0.7447
Epoch 3/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.5957 - accuracy: 0.7484 - val_loss: 0.5680 - val_accuracy: 0.7557
Epoch 4/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.5749 - accuracy: 0.7570 - val_loss: 0.5506 - val_accuracy: 0.7650
Epoch 5/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.5586 - accuracy: 0.7642 - val_loss: 0.5460 - val_accuracy: 0.7648
Epoch 6/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.5449 - accuracy: 0.7690 - val_loss: 0.5252 - val_accuracy: 0.7734
Epoch 7/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.5352 - accuracy: 0.7737 - val_loss: 0.5195 - val_accuracy:

##### In 4th one, I added a 64-nodes layer and also included a Dropout layer, but it just got worse

In [17]:
dict_inputs = feature_space.get_inputs()
encoded_features = feature_space.get_encoded_features()

x = keras.layers.Dense(16, activation="relu")(encoded_features)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(32, activation="relu")(x)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(64, activation="relu")(x)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(64, activation="relu")(x)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(64, activation="relu")(x)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(32, activation="relu")(x)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(16, activation="relu")(x)
# x = keras.layers.Dropout(0.3)(x)
predictions = keras.layers.Dense(8, activation="softmax")(x)

training_model = keras.Model(inputs=encoded_features, outputs=predictions)
training_model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

inference_model = keras.Model(inputs=dict_inputs, outputs=predictions)



training_model.fit(
    preprocessed_train_ds, epochs=20, validation_data=preprocessed_val_ds, verbose=1
)

Epoch 1/20
2542/2542 [==============================] - 5s 1ms/step - loss: 0.7542 - accuracy: 0.6856 - val_loss: 0.6396 - val_accuracy: 0.7251
Epoch 2/20
2542/2542 [==============================] - 4s 1ms/step - loss: 0.6334 - accuracy: 0.7363 - val_loss: 0.6083 - val_accuracy: 0.7477
Epoch 3/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.6000 - accuracy: 0.7499 - val_loss: 0.5907 - val_accuracy: 0.7544
Epoch 4/20
2542/2542 [==============================] - 4s 1ms/step - loss: 0.5794 - accuracy: 0.7577 - val_loss: 0.5507 - val_accuracy: 0.7649
Epoch 5/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.5615 - accuracy: 0.7674 - val_loss: 0.5433 - val_accuracy: 0.7700
Epoch 6/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.5501 - accuracy: 0.7720 - val_loss: 0.5272 - val_accuracy: 0.7787
Epoch 7/20
2542/2542 [==============================] - 4s 1ms/step - loss: 0.5393 - accuracy: 0.7748 - val_loss: 0.5193 - val_accuracy:

#### In 5th one, I added 2 more layers to complex model without any dropout layers, and got amazingly go 85% accuracy

In [18]:
dict_inputs = feature_space.get_inputs()
encoded_features = feature_space.get_encoded_features()

x = keras.layers.Dense(16, activation="relu")(encoded_features)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(32, activation="relu")(x)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(64, activation="relu")(x)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(128, activation="relu")(x)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(64, activation="relu")(x)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(32, activation="relu")(x)
# x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(16, activation="relu")(x)
# x = keras.layers.Dropout(0.3)(x)
predictions = keras.layers.Dense(8, activation="softmax")(x)

training_model = keras.Model(inputs=encoded_features, outputs=predictions)
training_model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

inference_model = keras.Model(inputs=dict_inputs, outputs=predictions)



training_model.fit(
    preprocessed_train_ds, epochs=20, validation_data=preprocessed_val_ds, verbose=1
)

Epoch 1/20
2542/2542 [==============================] - 4s 1ms/step - loss: 0.6802 - accuracy: 0.7156 - val_loss: 0.6193 - val_accuracy: 0.7328
Epoch 2/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.5872 - accuracy: 0.7505 - val_loss: 0.5744 - val_accuracy: 0.7535
Epoch 3/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.5492 - accuracy: 0.7664 - val_loss: 0.5479 - val_accuracy: 0.7635
Epoch 4/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.5210 - accuracy: 0.7776 - val_loss: 0.5146 - val_accuracy: 0.7768
Epoch 5/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.4977 - accuracy: 0.7884 - val_loss: 0.4950 - val_accuracy: 0.7874
Epoch 6/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.4760 - accuracy: 0.7971 - val_loss: 0.4773 - val_accuracy: 0.7958
Epoch 7/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.4597 - accuracy: 0.8065 - val_loss: 0.4632 - val_accuracy:

#### And my last model, which I just wanted to test if i keep complexity in a same level as reference just adding Dropout to descending part of model what would happen

In [19]:
dict_inputs = feature_space.get_inputs()
encoded_features = feature_space.get_encoded_features()

x = keras.layers.Dense(16, activation="relu")(encoded_features)
# x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(32, activation="relu")(x)
# x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(64, activation="relu")(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(64, activation="relu")(x)
# x = keras.layers.Dropout(0.5)(x)
x = keras.layers.concatenate([x,encoded_features])
x = keras.layers.Dense(32, activation="relu")(x)
# x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(16, activation="relu")(x)
# x = keras.layers.Dropout(0.2)(x)
predictions = keras.layers.Dense(8, activation="softmax")(x)

training_model = keras.Model(inputs=encoded_features, outputs=predictions)
training_model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

inference_model = keras.Model(inputs=dict_inputs, outputs=predictions)



training_model.fit(
    preprocessed_train_ds, epochs=20, validation_data=preprocessed_val_ds, verbose=1
)

Epoch 1/20
2542/2542 [==============================] - 4s 1ms/step - loss: 0.7012 - accuracy: 0.7071 - val_loss: 0.6153 - val_accuracy: 0.7350
Epoch 2/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.5967 - accuracy: 0.7463 - val_loss: 0.5765 - val_accuracy: 0.7522
Epoch 3/20
2542/2542 [==============================] - 4s 1ms/step - loss: 0.5633 - accuracy: 0.7612 - val_loss: 0.5471 - val_accuracy: 0.7680
Epoch 4/20
2542/2542 [==============================] - 4s 1ms/step - loss: 0.5372 - accuracy: 0.7745 - val_loss: 0.5315 - val_accuracy: 0.7778
Epoch 5/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.5170 - accuracy: 0.7812 - val_loss: 0.5068 - val_accuracy: 0.7865
Epoch 6/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.5020 - accuracy: 0.7874 - val_loss: 0.4855 - val_accuracy: 0.7945
Epoch 7/20
2542/2542 [==============================] - 3s 1ms/step - loss: 0.4882 - accuracy: 0.7944 - val_loss: 0.4845 - val_accuracy: